# 모든 데이터에서 위도경도 구하기

- 모든 데이터의 시군구, 번지의 결측치 크롤링
- 시군구, 번지 기준 위도와 경도 구하기
- 위도, 경도 기준으로 주변 50m, 100m, 500m, 1000m 기준 역과 버스정거정의 개수 구하기

- 좌표Y : 위도
- 좌표X : 경도

In [46]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r"/usr/share/fonts/truetype/nanum/NanumGothic.ttf", # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

from geopy.geocoders import Nominatim
import googlemaps

from haversine import haversine

In [47]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
train_path = '/root/data/train.csv'
test_path  = '/root/data/test.csv'
dt = pd.read_csv(train_path)
dt_test = pd.read_csv(test_path)

In [48]:
print('Train data shape : ', dt.shape, 'Test data shape : ', dt_test.shape)

Train data shape :  (1118822, 52) Test data shape :  (9272, 51)


In [222]:
dt_copy = dt_test.copy()

## 위도경도를 찾아야 하는 주소들만 추출하여 작업

In [66]:
import requests
a = dt_copy[dt_copy['좌표X'].isna()]
a['new_address'] = a['시군구'] + " " + a['도로명']
a['old_address'] = a['시군구'] + ' ' + a['번지']

In [68]:
need_tofind_newaddr = a[a['좌표X'].isna()]['new_address'].value_counts().index #위도 경도가 필요한 주소들
need_tofind_oldaddr = a[a['좌표X'].isna()]['old_address'].value_counts().index

In [87]:
need_tofind_newaddr

Index(['서울특별시 강북구 번동 덕릉로 111', '서울특별시 강북구 미아동 솔샘로 174',
       '서울특별시 송파구 신천동 올림픽로 435', '서울특별시 강동구 고덕동 고덕로 333',
       '서울특별시 관악구 봉천동 성현로 80', '서울특별시 서대문구 남가좌동 가재울미래로 2',
       '서울특별시 금천구 시흥동 금하로 816', '서울특별시 송파구 잠실동 올림픽로 135',
       '서울특별시 강동구 고덕동 아리수로50길 50', '서울특별시 성북구 돈암동 성북로4길 52',
       ...
       '서울특별시 도봉구 창동 노해로63길 75', '서울특별시 도봉구 창동 우이천로 200',
       '서울특별시 도봉구 창동 우이천로20길 7', '서울특별시 도봉구 창동 덕릉로 283',
       '서울특별시 도봉구 쌍문동 우이천로38가길 32', '서울특별시 도봉구 쌍문동 해등로25길 41',
       '서울특별시 도봉구 쌍문동 해등로25길 36', '서울특별시 도봉구 쌍문동 우이천로 328',
       '서울특별시 도봉구 쌍문동 해등로 190', '서울특별시 동작구 사당동 사당로2길 72'],
      dtype='object', length=2054)

In [119]:
lat_new = []
lon_new = []
is_find_new = []
lat_old = []
lon_old = []
is_find_old = []

In [120]:
# 신주소로 검색
tmp = 0
url = "https://dapi.kakao.com/v2/local/search/address.json" #요청할 url 주소
headers = {"Authorization": 'KakaoAK 7e13c0b7854960718bdc7832993d1a28'} #REST API 키(유효한 키)

for new_addr in need_tofind_newaddr:

    # if a.loc[idx, 'cant_find_xy'] == 1:
    #     continue

    
    query = {'query': new_addr } #입력할 주소
    result = requests.get(url,
                        headers=headers,
                        data=query).json() #카카오 API 요청
    try:
        lat_new.append(result['documents'][0]['y']) # 위도
        lon_new.append(result['documents'][0]['x']) # 경도
        is_find_new.append(None)
        print(new_addr + ' 성공')
    except:
        lat_new.append(None)
        lon_new.append(None)
        is_find_new.append(1)
        print(new_addr + ' 실패')

    # tmp += 1
    # if tmp > 99999:
    #     break

서울특별시 강북구 번동 덕릉로 111 성공
서울특별시 강북구 미아동 솔샘로 174 성공
서울특별시 송파구 신천동 올림픽로 435 성공
서울특별시 강동구 고덕동 고덕로 333 성공
서울특별시 관악구 봉천동 성현로 80 성공
서울특별시 서대문구 남가좌동 가재울미래로 2 성공
서울특별시 금천구 시흥동 금하로 816 성공
서울특별시 송파구 잠실동 올림픽로 135 성공
서울특별시 강동구 고덕동 아리수로50길 50 성공
서울특별시 성북구 돈암동 성북로4길 52 성공
서울특별시 마포구 아현동 마포대로 195 성공
서울특별시 중구 신당동 다산로 32 성공
서울특별시 강동구 암사동 고덕로 131 성공
서울특별시 강동구 암사동 상암로 11 성공
서울특별시 성동구 금호동1가 금호로 140 성공
서울특별시 강동구 상일동 고덕로 360 성공
서울특별시 노원구 상계동 한글비석로 479 성공
서울특별시 양천구 신정동 목동동로 130 성공
서울특별시 강남구 수서동 광평로19길 10 성공
서울특별시 구로구 신도림동 신도림로 16 성공
서울특별시 송파구 방이동 양재대로 1218 성공
서울특별시 송파구 가락동 동남로 193 성공
서울특별시 도봉구 도봉동 마들로 859-19 성공
서울특별시 양천구 신월동 남부순환로83길 18 성공
서울특별시 강서구 가양동 허준로 139 성공
서울특별시 도봉구 쌍문동 해등로 231 성공
서울특별시 도봉구 창동 덕릉로66길 17 성공
서울특별시 노원구 월계동 마들로 31 성공
서울특별시 송파구 잠실동 잠실로 62 성공
서울특별시 성동구 행당동 고산자로 164 성공
서울특별시 송파구 신천동 올림픽로35길 104 성공
서울특별시 성북구 장위동 한천로 713 성공
서울특별시 동대문구 휘경동 한천로 248 성공
서울특별시 양천구 신정동 신정로 293 성공
서울특별시 노원구 월계동 마들로 111 성공
서울특별시 강남구 개포동 개포로109길 9 성공
서울특별시 양천구 목동 목동동로 350 성공
서울특별시 구로구 개봉동 개봉로20길 6 성공
서울특별시 은평구 응암동 응암로30길

In [123]:
# 구주소로 검색
tmp = 0
url = "https://dapi.kakao.com/v2/local/search/address.json" #요청할 url 주소
headers = {"Authorization": 'KakaoAK 7e13c0b7854960718bdc7832993d1a28'} #REST API 키(유효한 키)

for old_addr in need_tofind_oldaddr:

    # if a.loc[idx, 'cant_find_xy'] == 1:
    #     continue

    
    query = {'query': old_addr } #입력할 주소
    result = requests.get(url,
                        headers=headers,
                        data=query).json() #카카오 API 요청
    try:
        lat_old.append(result['documents'][0]['y']) # 위도
        lon_old.append(result['documents'][0]['x']) # 경도
        is_find_old.append(None)
        print(old_addr + ' 성공')
    except:
        lat_old.append(None)
        lon_old.append(None)
        is_find_old.append(1)
        print(old_addr + ' 실패')

서울특별시 강북구 번동 443-6 성공
서울특별시 강북구 미아동 1353 성공
서울특별시 송파구 신천동 17 성공
서울특별시 강동구 고덕동 693 성공
서울특별시 관악구 봉천동 1712 성공
서울특별시 금천구 시흥동 1013 성공
서울특별시 성북구 돈암동 609-1 성공
서울특별시 강동구 고덕동 688 성공
서울특별시 송파구 잠실동 22 성공
서울특별시 마포구 아현동 777 성공
서울특별시 중구 신당동 844 성공
서울특별시 강동구 암사동 414-2 성공
서울특별시 노원구 상계동 639 성공
서울특별시 성동구 금호동1가 280 실패
서울특별시 노원구 월계동 13 성공
서울특별시 강동구 암사동 509 성공
서울특별시 강남구 개포동 12 성공
서울특별시 강동구 상일동 519 성공
서울특별시 양천구 신정동 329 성공
서울특별시 구로구 신도림동 642 성공
서울특별시 강남구 수서동 746 성공
서울특별시 동작구 사당동 105 성공
서울특별시 송파구 방이동 89 성공
서울특별시 송파구 가락동 140 성공
서울특별시 강서구 가양동 1475 성공
서울특별시 양천구 신월동 1076 성공
서울특별시 도봉구 도봉동 30-1 성공
서울특별시 노원구 월계동 18 성공
서울특별시 도봉구 창동 38 성공
서울특별시 도봉구 쌍문동 59 성공
서울특별시 성북구 장위동 173-114 실패
서울특별시 성동구 행당동 349 성공
서울특별시 송파구 잠실동 35 성공
서울특별시 서대문구 남가좌동 385 성공
서울특별시 동대문구 휘경동 57 성공
서울특별시 양천구 신정동 1259 성공
서울특별시 양천구 목동 912 성공
서울특별시 구로구 개봉동 481 성공
서울특별시 노원구 상계동 691 성공
서울특별시 은평구 응암동 767 성공
서울특별시 양천구 신정동 311 성공
서울특별시 관악구 봉천동 1718 성공
서울특별시 노원구 상계동 670 성공
서울특별시 서대문구 남가좌동 376 성공
서울특별시 성동구 행당동 347 성공
서울특별시 도봉구 방학동 271-1 성공
서울특별시 영등포구 대림동 608-1

In [124]:
addr_list_new = {
    'new_addr' : need_tofind_newaddr,
    'lat' : lat_new,
    'lon' : lon_new,
    'is_find' : is_find_new
    }

In [137]:
addr_list_old = {
    'old_addr' : need_tofind_oldaddr,
    'lat' : lat_old,
    'lon' : lon_old,
    'is_find' : is_find_old
    }

In [138]:
found_newaddr = pd.DataFrame(addr_list_new)
found_oldaddr = pd.DataFrame(addr_list_old)

In [127]:
found_newaddr

,new_addr,lat,lon,is_find
0,서울특별시 강북구 번동 덕릉로 111,37.6351210518367,127.026056960118,NaN
1,서울특별시 강북구 미아동 솔샘로 174,37.6176383628418,127.010599691076,NaN
2,서울특별시 송파구 신천동 올림픽로 435,37.5213592954462,127.104971745978,NaN
3,서울특별시 강동구 고덕동 고덕로 333,37.5561770852501,127.159176546352,NaN
4,서울특별시 관악구 봉천동 성현로 80,37.4898474387668,126.952985510916,NaN
...,...,...,...,...
2049,서울특별시 도봉구 쌍문동 해등로25길 41,37.6558809051642,127.034124384435,NaN
2050,서울특별시 도봉구 쌍문동 해등로25길 36,37.6557984051554,127.03175856369,NaN
2051,서울특별시 도봉구 쌍문동 우이천로 328,37.6468785281218,127.02881783318,NaN
2052,서울특별시 도봉구 쌍문동 해등로 190,37.6582166673064,127.035759433408,NaN


In [128]:
found_newaddr[found_newaddr['is_find']==1]

,new_addr,lat,lon,is_find
257,서울특별시 강남구 도곡동 남부순환로 2803,None,None,1.0
810,서울특별시 서초구 방배동 효령로34길 79,None,None,1.0
821,서울특별시 서초구 반포동 신반포로 9,None,None,1.0
1116,서울특별시 중랑구 상봉동 신내로7길 20,None,None,1.0
1658,서울특별시 강서구 방화동 방화대로34길 110,None,None,1.0
1659,서울특별시 강서구 방화동 방화대로34길 62,None,None,1.0
1853,서울특별시 서초구 방배동 효령로21길 47,None,None,1.0
2007,서울특별시 동작구 노량진동 만양로 26,None,None,1.0


In [139]:
found_oldaddr

,old_addr,lat,lon,is_find
0,서울특별시 강북구 번동 443-6,37.6351692502231,127.026076576838,NaN
1,서울특별시 강북구 미아동 1353,37.6179760334621,127.01263074872,NaN
2,서울특별시 송파구 신천동 17,37.5205366173754,127.10615089337,NaN
3,서울특별시 강동구 고덕동 693,37.5580690838901,127.163465686409,NaN
4,서울특별시 관악구 봉천동 1712,37.489767905811,126.950619822698,NaN
...,...,...,...,...
2010,서울특별시 도봉구 창동 635-42,37.6414406910731,127.034034160072,NaN
2011,서울특별시 도봉구 창동 442-33,37.6424106463897,127.041234691696,NaN
2012,서울특별시 도봉구 쌍문동 719,37.6523831576565,127.02125900287,NaN
2013,서울특별시 도봉구 쌍문동 158,37.6560145987519,127.033853936885,NaN


In [140]:
found_oldaddr[found_oldaddr['is_find']==1]

,old_addr,lat,lon,is_find
13,서울특별시 성동구 금호동1가 280,None,None,1.0
30,서울특별시 성북구 장위동 173-114,None,None,1.0
128,서울특별시 강남구 개포동 138,None,None,1.0
322,서울특별시 서대문구 북아현동 149,None,None,1.0
640,서울특별시 구로구 항동 가-238,None,None,1.0
807,서울특별시 관악구 신림동 1644,None,None,1.0
1050,서울특별시 중랑구 상봉동 785,None,None,1.0
1392,서울특별시 관악구 봉천동 1152,None,None,1.0
1446,서울특별시 구로구 고척동 97,None,None,1.0


## 주소별 위도 경도 데이터를 실제 데이터에 대입

In [225]:
dt_copy['new_address'] = dt_copy['시군구'] + " " + dt_copy['도로명']
dt_copy['old_address'] = dt_copy['시군구'] + " " + dt_copy['번지']
dt_copy['cant_find_xy'] = np.nan
dt_copy

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,new_address,old_address,cant_find_xy
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.9700,202307,26,5,1987,...,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,서울특별시 강남구 개포동 언주로 3,서울특별시 강남구 개포동 658-1,NaN
1,서울특별시 강남구 개포동,651-1,651.0,1.0,개포더샵트리에,108.2017,202308,15,10,2021,...,의무,2022-02-23 13:01:10.0,Y,N,127.056394,37.484892,2022-02-23 11:05:05.0,서울특별시 강남구 개포동 개포로 311,서울특별시 강남구 개포동 651-1,NaN
2,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,161.0000,202307,28,15,1984,...,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,서울특별시 강남구 개포동 개포로 307,서울특별시 강남구 개포동 652,NaN
3,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,133.4600,202308,10,14,1984,...,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,서울특별시 강남구 개포동 개포로 307,서울특별시 강남구 개포동 652,NaN
4,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,104.4300,202308,18,6,1984,...,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,서울특별시 강남구 개포동 개포로 307,서울특별시 강남구 개포동 652,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9267,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.6500,202307,19,13,2014,...,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,서울특별시 중랑구 신내동 신내역로1길 85,서울특별시 중랑구 신내동 816,NaN
9268,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.6200,202307,25,12,2014,...,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,서울특별시 중랑구 신내동 신내역로1길 85,서울특별시 중랑구 신내동 816,NaN
9269,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,101.6500,202308,27,12,2014,...,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,서울특별시 중랑구 신내동 신내역로1길 85,서울특별시 중랑구 신내동 816,NaN
9270,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.9400,202309,2,18,2014,...,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,서울특별시 중랑구 신내동 신내역로1길 85,서울특별시 중랑구 신내동 816,NaN


In [226]:
# 신주소기준 위도 경도 갱신
for adr, lat, lon, is_find in zip(found_newaddr['new_addr'],found_newaddr['lat'], found_newaddr['lon'], found_newaddr['is_find']):
    # 만약 해당 주소의 위도 경도를 찾았다면 
    
    if is_find == 1:
        dt_copy.loc[(dt_copy['new_address'] == adr) & (dt_copy['좌표X'].isna()), ['cant_find_xy'] ] = is_find
    else:
        dt_copy.loc[(dt_copy['new_address'] == adr) & (dt_copy['좌표X'].isna()), ['좌표X'] ] = lon
        dt_copy.loc[(dt_copy['new_address'] == adr) & (dt_copy['좌표Y'].isna()), ['좌표Y'] ] = lat
    # print(adr, lat, lon, is_find)

In [229]:
# 구주소기준 위도 경도 갱신
for adr, lat, lon, is_find in zip(found_oldaddr['old_addr'],found_oldaddr['lat'], found_oldaddr['lon'], found_oldaddr['is_find']):
    # 만약 해당 주소의 위도 경도를 찾았다면 
    if is_find == 1:
        dt_copy.loc[(dt_copy['old_address'] == adr) & dt_copy['좌표X'].isna(), ['cant_find_xy'] ] = is_find
    else:
        dt_copy.loc[(dt_copy['old_address'] == adr) & dt_copy['좌표X'].isna(), ['좌표X'] ] = lon
        dt_copy.loc[(dt_copy['old_address'] == adr) & dt_copy['좌표Y'].isna(), ['좌표Y'] ] = lat

In [230]:
dt_copy[dt_copy['좌표X'].isna()].isna()

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,new_address,old_address,cant_find_xy
9166,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,False,False,False


## 최종본 저장

In [231]:
dt_copy.to_csv('/root/data/make_lat_lon_intest.csv', index=False)